In [1]:
# SKT Brain github 주소는 다음과 같습니다. https://github.com/SKTBrain/KoBERT

# !pip install mxnet
# !pip install gluonnlp pandas tqdm
# !pip install sentencepiece
# !pip install transformers==3 # 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
# !pip install torch

# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

# kobert 
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

# transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup


In [2]:
# GPU 사용 시
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Bert모델, Voca 불러오기
bertmodel, vocab = get_pytorch_kobert_model()


using cached model
using cached model


In [3]:
# 학습용 데이터셋 불러오기
import pandas as pd
# !wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1

# !wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

In [4]:
#field_indices=[데이터 있는 열, 레이블링 된 열]
#num_discard_samples=헤더가 있으면 상위 row 1개 제거
# df_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2],num_discard_samples=1)
# df_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2],num_discard_samples=1)

In [5]:
# df_train

In [6]:
# # 질병명 라벨링
# from sklearn.preprocessing import LabelEncoder
# encoder = LabelEncoder()
# encoder.fit(new_data['질병명'])
# new_data['질병명'] = encoder.transform(new_data['질병명'])
# new_data.head()

# # 라벨링된 질병명 매핑 ex) {0: '비염', 1: '소화불량', 2: '수족냉증', 3: '식중독', 4: '질염'}
# mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
# mapping

In [7]:
PATH = '../challenge/'

In [43]:
new_data = pd.read_csv(PATH + "Clean_train.csv", encoding='euc-kr')
new_data.sample(n=10)

,title,cls
29,김은별 일 현지시간 유럽 주요 증시는 하락 출발했다 개장 직후 영국 FTSE 지수...,0
33,전작 신뢰에 양호한 수익률로 자금 유입 활발 KB중소형주포커스 올 억 자금몰이 형만...,1
25,CJ오쇼핑 GS홈쇼핑 하이트진로 롯데삼강 포스코ICT 에스원 등이 지난해 사상 처음...,1
48,LG생활건강이 수익성을 회복할지 주목된다 일 금융투자업계에 따르면 LG생활건강의 올...,1
88,한화 가 신정부 관련주로 뜨고 있다 그룹 내 주력사업인 방위산업과 항공우주산업 보험...,1
122,유럽증시가 지난달 일 현지시간 이틀 연속 상승세를 이어갔다 이탈리아 총선 결과 충격...,1
32,태블릿 PC 등 활용한 아웃도어 세일즈 잇달아 신시장 개척 경비절감 효과 국내 증권...,1
68,미국 연방 정부의 예산이 자동으로 삭감되는 이른바 시퀘스터 sequester 가 결...,1
16,코웨이 솔고바이오 등 월에 상승 봄철 계절 테마주라 할 수 있는 황사 관련주들이 올...,0
43,헤럴드생생뉴스 미국 민주당과 공화당이 시퀘스터 회피를 위해 제출안 법안이 모두 부결...,1


In [44]:
data_list = []
for q, label in zip(new_data['title'], new_data['cls'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [45]:
print(data_list[0])
print(data_list[60])

['뉴욕 이상원 특파원 일 현지시간 뉴욕증시는 미국의 경제 지표 부진과 연방 정부의 예산 자동 삭감 우려에 하락세로 출발했다 이날 오전 시 분 현재 뉴욕증권거래소 NYSE 에서 다우존스 산업평균지수는 전날보다 포인트 떨어진 에서 거래되고 있다 스탠더드 앤드 푸어스 S P 지수는 포인트 낮은 을 나스닥 종합지수는 포인트 내려간 을 각각 기록하고 있다 시장은 미국 연방 정부의 예산 자동 삭감을 일컫는 시퀘스터 협상에 주목하고 있다 버락 오바마 대통령은 시퀘스터가 발동하는 이날 존 베이너 공화당 하원 의장 등 의회 지도부와 시퀘스터 돌파구 마련을 위한 회의를 한다 미국 소비자들의 개인소득은 비교적 큰 폭으로 줄었으나 지출은 증가세를 이어갔다 미국 상무부는 지난 월 소비지출이 전월보다 늘어나 지난해 월 이후 개월 연속 증가했다고 밝혔다 이는 시장전문가들의 예상치 평균과 일치하는 증가 폭이다 그러나 같은 달 개인소득은 줄어들어 지난 년 월 이후 년 만에 최대 감소폭을 기록했다 뉴욕상업거래소 NYMEX 에서 서부텍사스산 원유 WTI 는 전날보다 달러 떨어진 배럴당 달러 선에서 움직이고 있다 ', '0']
[' 김재연 중국의 월 HSBC 제조업 구매관리자지수 PMI 수정치가 를 기록했다고 블룸버그통신이 일 보도했다 이는 시장 예상치 는 물론 전월치 을 하회한 것이다 ', '0']


In [46]:
type(data_list[0][1])

str

In [47]:

# Train / Test set 분리
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=42)
print("train shape is:", len(dataset_train))
print("test shape is:", len(dataset_test))


train shape is: 99
test shape is: 25


In [48]:
# kobert 입력 데이터로 만들기

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [49]:
# Setting parameters
max_len = 64 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10 # 훈련 반복횟수
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [50]:
# 토큰화
# 기본 Bert tokenizer 사용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [51]:
# data 패딩화
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [52]:
data_train[0]
#첫 번째는 패딩된 시퀀스
#두 번째는 길이와 타입에 대한 내용
#세 번재는 어텐션 마스크 시퀀스

(array([   2, 1146, 7096, 5012, 7645, 6705, 5136, 7207, 5379, 6116,  517,
        7802, 2277, 6903, 2332, 1235, 5439, 7087, 6896, 3468, 7941, 7207,
        7003,  517, 5566, 5436, 7227, 7088, 1938, 2280, 4555, 7295, 7139,
        3803, 2357, 5760, 1146, 5566, 7095, 1235, 5439, 7087, 5859, 3468,
        7941, 5436, 7227, 7088, 2779, 5330,  742, 5760, 1222, 7253, 7846,
        7088, 1614, 1146, 6335, 3004, 6077, 3811,  851,    3], dtype=int32),
 array(64, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [53]:
# torch 형식의 dataset 만들어주기 (pytorch용 DataLoader 사용)
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [54]:
# Kobert 학습모델 만들기

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 2, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)
      

In [55]:
#conda install pytorch torchvision cudatoolkit=10.1 -c pytorch

In [56]:
import torch
print(torch.cuda.device_count())

2


In [57]:
# Bert 모델 불러오기

model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# optimizer설정 하고 schedule 설정(linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


In [58]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
  
train_dataloader

In [59]:
# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    

/anaconda/envs/azureml_py36_pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/2 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7736221551895142 train acc 0.46875
epoch 1 train acc 0.5200892857142857


/anaconda/envs/azureml_py36_pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1 [00:00<?, ?it/s]

...

epoch 9 test acc 0.84


  0%|          | 0/2 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.22332027554512024 train acc 0.953125
epoch 10 train acc 0.9622767857142858


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 10 test acc 0.84


In [65]:
# 테스트 문장 예측
test_sentence = "본격적으로 막이 오른 쌍용차 인수전이 SM그룹의 '깜짝 등판'으로 후끈 달아오른 가운데 당사자인 쌍용차 역시 예상외의 흥행에 미소 지으며 후속 작업을 준비하고 있다. 최근 쌍용차가 친환경차 전용공장 건설 계획을 밝히는 등 전기차 전환에 뒤늦게 속도를 내는 가운데 상당수의 투자자가 전기차 사업 확대를 목표로 인수 의향을 밝히고 나선 것도 긍정적이라는 분위기다."
test_label = 1 # 실제 값

unseen_test = pd.DataFrame([[test_sentence, test_label]], columns = [['title', 'cls']])
unseen_values = unseen_test.values
test_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=5)

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    out = model(token_ids, valid_length, segment_ids)
    print(out)
    
model.eval() # 평가 모드로 변경
    
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/anaconda/envs/azureml_py36_pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


  0%|          | 0/1 [00:00<?, ?it/s]

tensor([[-0.8443,  1.2456]], device='cuda:0', grad_fn=<AddmmBackward>)


/anaconda/envs/azureml_py36_pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 10 test acc 2.52


In [ ]:
# 감성값 예측하는 함수 만들기

import pandas as pd

# 위에서 설정한 tok, max_len, batch_size, device를 그대로 입력
# comment : 예측하고자 하는 텍스트 데이터 리스트
def getSentimentValue(comment, tok, max_len, batch_size, device):
    commnetslist = [] # 텍스트 데이터를 담을 리스트
    emo_list = [] # 감성 값을 담을 리스트
    for c in comment: # 모든 댓글
        commnetslist.append( [c, 5] ) # [댓글, 임의의 양의 정수값] 설정

    pdData = pd.DataFrame( commnetslist, columns = [['댓글', '감성']] )
    pdData = pdData.values
    test_set = BERTDataset(pdData, 0, 1, tok, max_len, True, False) 
    test_input = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=5)

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_input):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length 
        # 이때, out이 예측 결과 리스트
        out = model(token_ids, valid_length, segment_ids)

        # e는 2가지 실수 값으로 구성된 리스트
        # 0번 인덱스가 더 크면 부정, 긍정은 반대
        for e in out:
            if e[0]>e[1]: # 부정
            value = 0
            else: #긍정
            value = 1
            emo_list.append(value)

    return emo_list # 텍스트 데이터에 1대1 매칭되는 감성값 리스트 반환

# input : 텍스트 데이터 리스트 외 KoBERT 설정 파라미터들
# output : 입력한 텍스트 데이터 리스트와 1대 1 매칭 되는 감성 값 리스트